In [17]:
import tkinter as tk
import tkinter.ttk as ttk
from tkinter.filedialog import askopenfilename, asksaveasfilename
import numpy as np
import soundfile as sf
import sounddevice as sd
import struct

# --- Helper functions to read/write WAV headers for encoded audio ---

def write_encoded_wav(filename, data, samplerate):
    # data: numpy int32 array
    byte_data = struct.pack('<' + 'i' * len(data), *data)
    with open(filename, 'wb') as out:
        out.write(struct.pack('<4sI4s', b'RIFF', 36 + len(byte_data), b'WAVE'))
        # Format chunk: PCM(1), mono(1), 32 bits/sample
        out.write(struct.pack('<4sIHHIIHH', b'fmt ', 16, 1, 1, samplerate,
                              samplerate * 4, 4, 32))
        out.write(struct.pack('<4sI', b'data', len(byte_data)))
        out.write(byte_data)

def read_encoded_wav(filename):
    with open(filename, 'rb') as f:
        # Read header chunks (simplified)
        riff = f.read(12)
        fmt = f.read(24)
        data_header = f.read(8)
        # Extract sample rate and data size
        samplerate = struct.unpack('<I', fmt[12:16])[0]
        datasize = struct.unpack('<I', data_header[4:8])[0]
        data = f.read(datasize)
        samples = struct.unpack('<' + 'i' * (datasize // 4), data)
        return np.array(samples, dtype=np.int32), samplerate

# --- Audio class with audio + sample rate ---

class Audio(tk.Variable):
    def __init__(self):
        super().__init__()
        self.audio = None  # numpy array
        self.fm = -1      # sample rate

    def get(self):
        return self.audio, self.fm

    def set(self, audio, fm):
        self.audio = audio
        self.fm = fm

# --- File selection button ---

def selecAudio(raiz, varAudio):
    varMensaje = tk.StringVar(raiz, 'Selecciona una señal de audio')
    def selecAudio():
        fichero = askopenfilename(title='Selecciona un fichero de audio',
                                filetypes=(('WAV Files', '*.wav'),
                                           ('Todos los archivos', '*.*')))
        if fichero:
            varMensaje.set(fichero)
            data, samplerate = sf.read(fichero, dtype='float32')
            varAudio.set(data, samplerate)
    boton = ttk.Button(raiz, textvariable=varMensaje, command=selecAudio)
    boton.pack(pady=5)

# --- Playback and control buttons ---

def reproduce(varAudio, varVolumen):
    audio, fm = varAudio.get()
    if audio is not None:
        sd.play(audio * varVolumen.get(), fm, blocking=False)

def reproduceParaSal(raiz, varAudio, varVolumen):
    marco = ttk.Frame(raiz)
    marco.pack(side=tk.BOTTOM, pady=5)

    repro = ttk.Button(marco, text='Play', command=lambda: reproduce(varAudio, varVolumen))
    repro.pack(side=tk.LEFT, padx=5)

    parar = ttk.Button(marco, text='Stop', command=sd.stop)
    parar.pack(side=tk.LEFT, padx=5)

    salir = ttk.Button(marco, text='Quit', command=raiz.destroy)
    salir.pack(side=tk.LEFT, padx=5)

    return repro  # return Play button to enable/disable if needed

# --- Volume control ---

def volumen(raiz, varVolumen):
    marco = ttk.LabelFrame(raiz, text='Volumen')
    marco.pack()
    volumen = ttk.Scale(marco, from_=0, to=1, length=200, value=varVolumen.get(),
                       variable=varVolumen)
    volumen.pack()

# --- Extra buttons for each tab ---

def botonExtra1(raiz, varAudio, varOutput, num):
    def repartir():
        if num == 1:
            funModif_1(varAudio, varOutput)
        elif num == 2:
            funModif_2(varAudio, varOutput)
        elif num == 3:
            funModif_3(varAudio, varOutput)
        else:
            funModif_4(varAudio, varOutput)

    text_map = {
        1: 'Aplicar Mono',
        2: 'Aplicar Stereo',
        3: 'Codificar Stereo',
        4: 'Decodificar Stereo',
    }

    ttk.Button(
        raiz,
        text=text_map.get(num, 'Aplicar Modificación'),
        command=repartir
    ).pack(side=tk.LEFT, padx=10)

def botonExtra2(raiz, varOutput):
    def guardar():
        audio, fm = varOutput.get()
        if audio is None:
            return
        fichero = asksaveasfilename(defaultextension='.wav',
                                    filetypes=[('WAV files', '*.wav'), ('All files', '*.*')])
        if fichero:
            # Determine if data is encoded or normal by dtype and shape
            if audio.dtype == np.int32 and audio.ndim == 1:
                # encoded audio
                write_encoded_wav(fichero, audio, fm)
            else:
                # normal audio save
                sf.write(fichero, audio, fm)
    ttk.Button(
        raiz,
        text='Guardar Audio',
        command=guardar
    ).pack(side=tk.LEFT, padx=10)

# --- Audio modification functions ---

def funModif_1(varAudio, varOutput):
    # Stereo to Mono (simple average of channels)
    audio, fm = varAudio.get()
    if audio is None:
        return
    if audio.ndim == 1:
        # Already mono
        varOutput.set(audio, fm)
        return
    if audio.shape[1] == 2:
        mono = audio.mean(axis=1)
        varOutput.set(mono, fm)
    else:
        varOutput.set(audio, fm)

def funModif_2(varAudio, varOutput):
    # Mono to Stereo (duplicate mono channel)
    audio, fm = varAudio.get()
    if audio is None:
        return
    if audio.ndim == 1:
        stereo = np.column_stack((audio, audio))
        varOutput.set(stereo, fm)
    else:
        varOutput.set(audio, fm)

def funModif_3(varAudio, varOutput):
    # Encode stereo audio into int32 coded mono
    audio, fm = varAudio.get()
    if audio is None:
        return
    if audio.ndim != 2 or audio.shape[1] != 2:
        print("Error: input must be stereo audio.")
        return

    int_audio = (audio * 32767).astype(np.int16)
    L = int_audio[:, 0].astype(np.int32)
    R = int_audio[:, 1].astype(np.int32)
    codificats = ((L + R) << 16) | ((L - R) & 0xFFFF)
    varOutput.set(codificats, fm)

def funModif_4(varAudio, varOutput):
    # Decode int32 coded mono into stereo audio
    encoded, fm = varAudio.get()
    if encoded is None:
        return
    if not (encoded.dtype == np.int32 and encoded.ndim == 1):
        print("Error: input must be encoded int32 array.")
        return

    def signed_16bit(val):
        return val if val < 0x8000 else val - 0x10000

    vec_signed_16bit = np.vectorize(signed_16bit)
    lower_16 = encoded & 0xFFFF
    lower_16_signed = vec_signed_16bit(lower_16)
    upper_16 = encoded >> 16

    L = ((upper_16 + lower_16_signed) // 2).astype(np.int16)
    R = ((upper_16 - lower_16_signed) // 2).astype(np.int16)
    stereo = np.column_stack((L, R))
    stereo_float = stereo.astype(np.float32) / 32767.0
    varOutput.set(stereo_float, fm)

# --- Tab content creator ---

def crearTabContenido(tabControl, num):
    tab = ttk.Frame(tabControl)
    varAudio = Audio()
    varOutput = Audio()
    varVolumen = tk.DoubleVar(tabControl, 0.6)

    selecAudio(tab, varAudio)
    botonExtra1(tab, varAudio, varOutput, num)
    botonExtra2(tab, varOutput)
    vol_frame = ttk.Frame(tab)
    vol_frame.pack()
    volumen(vol_frame, varVolumen)
    play_btn = reproduceParaSal(tab, varOutput, varVolumen)

    # Disable play button for encoded audio (tab 3)
    if num == 3:
        play_btn.state(['disabled'])
    else:
        play_btn.state(['!disabled'])

    return tab

# --- Main ---

def main():
    win = tk.Tk()
    win.title('Reproductor audio de senyals - v4 Completo')

    tabControl = ttk.Notebook(win)
    tabs = []
    for i in range(1, 5):
        tab = crearTabContenido(tabControl, i)
        tabControl.add(tab, text=f'TAB {i}')
        tabs.append(tab)
    tabControl.pack(expand=1, fill='both')

    win.mainloop()

if __name__ == '__main__':
    main()


In [5]:
import tkinter as tk
from tkinter import ttk
from tkinter.filedialog import askopenfilename
from tkinter.filedialog import asksaveasfilename
import soundfile as sf
import sounddevice as sd

class Audio(tk.Variable):
    def __init__(self):
        self.audio = None
        self.fm = -1

    def get(self):
        return self.audio, self.fm

    def set(self, audio, fm):
        self.audio = audio
        self.fm = fm

def selecAudio(raiz, varAudio):
    varMensaje = tk.StringVar(raiz, 'Selecciona una señal de audio')

    def abrirArchivo():
        fichero = askopenfilename(
            title='Selecciona un fichero de audio',
            filetypes=(('Fichero de audio', '.mp3 .wav'), ('Todos los archivos', '*.*'))
        )
        if fichero:
            varMensaje.set(fichero)
            senyal, fm = sf.read(fichero)
            varAudio.set(senyal, fm)

    boton = ttk.Button(raiz, textvariable=varMensaje, command=abrirArchivo)
    boton.pack()

def reproduce(varAudio, varVolumen):
    audio, fm = varAudio.get()
    if audio is not None:
        sd.play(audio * varVolumen.get(), fm, blocking=False)

def reproduceParaSal(raiz, varAudio, varVolumen):
    marco = ttk.Frame(raiz)
    marco.pack(side=tk.BOTTOM)

    ttk.Button(marco, text='Play', command=lambda: reproduce(varAudio, varVolumen)).pack(side=tk.LEFT)
    ttk.Button(marco, text='Stop', command=sd.stop).pack(side=tk.LEFT)
    ttk.Button(marco, text='Quit', command=raiz.quit).pack(side=tk.LEFT)

def volumen(raiz, varVolumen):
    marco = ttk.LabelFrame(raiz, text='Volumen')
    marco.pack()
    ttk.Scale(marco, from_=0, to=1, length=200, variable=varVolumen).pack()

# === New buttons ===
def botonExtra1(raiz, tab, varAudio, varOutput, num):
    def repartir(raiz, varAudio, varOutput, num):
        if num == 1:
            funModif_1(raiz, varAudio, varOutput)
        elif num == 2:
            funModif_2(raiz, varAudio, varOutput)
        elif num == 3:
            funModif_3(raiz, varAudio, varOutput)
        else:
            funModif_4(raiz, varAudio, varOutput)

    # Set the button label depending on num
    titulos = {
        1: 'Filtrar Ruido',
        2: 'Normalizar Volumen',
        3: 'Invertir Señal',
        4: 'Aplicar Eco'
    }
    texto_boton = titulos.get(num, 'Aplicar Modificación')

    ttk.Button(
        raiz,
        text=texto_boton,
        command=lambda: repartir(tab, varAudio, varOutput, num)
    ).pack(side=tk.LEFT, padx=10)


def botonExtra2(raiz, varOutput):
    def guardarAudio():
        audio, fm = varOutput.get()
        if audio is not None:
            archivo = asksaveasfilename(
                title='Guardar archivo de audio',
                defaultextension='.wav',
                filetypes=(('Archivo WAV', '*.wav'), ('Todos los archivos', '*.*'))
            )
            if archivo:
                sf.write(archivo, audio, fm)
                print(f'Audio guardado en: {archivo}')
        else:
            print("No hay audio para guardar.")

    ttk.Button(raiz, text='Guardar Audio', command=guardarAudio).pack(side=tk.LEFT)

# === Audio player layout per tab ===
def crearTabContenido(tab, num):
    varAudio = Audio()
    varOutput = Audio()
    varVolumen = tk.DoubleVar(tab, 0.6)

    selecAudio(tab, varAudio)

    # Botones extra
    marcoBotonesExtra = ttk.Frame(tab)
    marcoBotonesExtra.pack(pady=10)
    botonExtra1(marcoBotonesExtra, tab, varAudio, varOutput, num)  # Updated
    botonExtra2(marcoBotonesExtra, varOutput)

    volumen(tab, varVolumen)
    reproduceParaSal(tab, varAudio, varVolumen)

# === Main app ===
def main():
    root = tk.Tk()
    root.title("Notebook con Reproductores de Audio")
    root.geometry("600x300")
    root.configure(background='Lavender Blush')
    ttk.Style().configure('.', font=("Arial", 14), background='Lavender Blush', padding=8)

    notebook = ttk.Notebook(root)
    notebook.pack(expand=True, fill='both')

    for i in range(4):
        tab = ttk.Frame(notebook)
        notebook.add(tab, text=f'Tab {i+1}')
        crearTabContenido(tab, num=i + 1)  # Pass tab number



    root.mainloop()

if __name__ == '__main__':
    main()


In [ ]:
import tkinter as tk
from tkinter import ttk
from tkinter.filedialog import askopenfilename
from tkinter.filedialog import asksaveasfilename
import soundfile as sf
import sounddevice as sd

class Audio(tk.Variable):
    def __init__(self):
        self.audio = None
        self.fm = -1

    def get(self):
        return self.audio, self.fm

    def set(self, audio, fm):
        self.audio = audio
        self.fm = fm

def selecAudio(raiz, varAudio):
    varMensaje = tk.StringVar(raiz, 'Selecciona una señal de audio')

    def abrirArchivo():
        fichero = askopenfilename(
            title='Selecciona un fichero de audio',
            filetypes=(('Fichero de audio', '.mp3 .wav'), ('Todos los archivos', '*.*'))
        )
        if fichero:
            varMensaje.set(fichero)
            senyal, fm = sf.read(fichero)
            varAudio.set(senyal, fm)

    boton = ttk.Button(raiz, textvariable=varMensaje, command=abrirArchivo)
    boton.pack()

def reproduce(varAudio, varVolumen):
    audio, fm = varAudio.get()
    if audio is not None:
        sd.play(audio * varVolumen.get(), fm, blocking=False)

def reproduceParaSal(raiz, varAudio, varVolumen):
    marco = ttk.Frame(raiz)
    marco.pack(side=tk.BOTTOM)

    ttk.Button(marco, text='Play', command=lambda: reproduce(varAudio, varVolumen)).pack(side=tk.LEFT)
    ttk.Button(marco, text='Stop', command=sd.stop).pack(side=tk.LEFT)
    ttk.Button(marco, text='Quit', command=raiz.quit).pack(side=tk.LEFT)

def volumen(raiz, varVolumen):
    marco = ttk.LabelFrame(raiz, text='Volumen')
    marco.pack()
    ttk.Scale(marco, from_=0, to=1, length=200, variable=varVolumen).pack()

def EstereoMono(varAudio, varOutput, canal=2):
    import numpy as np

    audio, fm = varAudio.get()
    if audio is None:
        return  # No input audio

    # Confirm it's stereo
    if audio.ndim != 2 or audio.shape[1] != 2:
        raise ValueError("La señal de entrada no es estéreo.")

    L = audio[:, 0]
    R = audio[:, 1]

    if canal == 0:
        mono = L
    elif canal == 1:
        mono = R
    elif canal == 2:
        mono = (L + R) / 2
    elif canal == 3:
        mono = (L - R) / 2
    else:
        raise ValueError("Canal debe ser 0, 1, 2 o 3.")

    # Save mono as 2D array for sounddevice compatibility
    mono = mono.reshape(-1, 1)
    varOutput.set(mono, fm)

def MonoEstereo(varAudio, varOutput):
    import numpy as np

    audio, fm = varAudio.get()
    if audio is None:
        return  # No input audio

    # If already stereo, just pass through or raise error
    if audio.ndim == 2 and audio.shape[1] == 2:
        raise ValueError("La señal ya es estéreo.")

    # Make sure audio is 1D for mono
    if audio.ndim == 2 and audio.shape[1] == 1:
        audio = audio[:, 0]

    # Duplicate mono to stereo by stacking channels
    stereo = np.column_stack((audio, audio))

    varOutput.set(stereo, fm)


def botonExtra1(raiz, varAudio, varOutput, num):
    def repartir(varAudio, varOutput, num):
        if num == 1:
            EstereoMono(varAudio, varOutput)
        elif num == 2:
            MonoEstereo(varAudio, varOutput)
        elif num == 3:
            funModif_3(varAudio, varOutput)
        else:
            funModif_4(varAudio, varOutput)

    titulos = {
        1: 'Filtrar Ruido',
        2: 'Normalizar Volumen',
        3: 'Invertir Señal',
        4: 'Aplicar Eco'
    }
    texto_boton = titulos[num]

    ttk.Button(
        raiz,
        text=texto_boton,
        command=lambda: repartir(varAudio, varOutput, num)
    ).pack(side=tk.LEFT, padx=10)



def botonExtra2(raiz, varOutput):
    def guardarAudio():
        audio, fm = varOutput.get()
        if audio is not None:
            archivo = asksaveasfilename(
                title='Guardar archivo de audio',
                defaultextension='.wav',
                filetypes=(('Archivo WAV', '*.wav'), ('Todos los archivos', '*.*'))
            )
            if archivo:
                sf.write(archivo, audio, fm)
                print(f'Audio guardado en: {archivo}')
        else:
            print("No hay audio para guardar.")

    ttk.Button(raiz, text='Guardar Audio', command=guardarAudio).pack(side=tk.LEFT)

# === Audio player layout per tab ===
def crearTabContenido(tab, num):
    varAudio = Audio()
    varOutput = Audio()
    varVolumen = tk.DoubleVar(tab, 0.6)

    selecAudio(tab, varAudio)

    # Botones extra
    marcoBotonesExtra = ttk.Frame(tab)
    marcoBotonesExtra.pack(pady=10)
    botonExtra1(marcoBotonesExtra, tab, varAudio, varOutput, num)  # Updated
    botonExtra2(marcoBotonesExtra, varOutput)

    volumen(tab, varVolumen)
    reproduceParaSal(tab, varAudio, varVolumen)

# === Main app ===
def main():
    root = tk.Tk()
    root.title("Notebook con Reproductores de Audio")
    root.geometry("600x300")
    root.configure(background='Lavender Blush')
    ttk.Style().configure('.', font=("Arial", 14), background='Lavender Blush', padding=8)

    notebook = ttk.Notebook(root)
    notebook.pack(expand=True, fill='both')

    for i in range(4):
        tab = ttk.Frame(notebook)
        notebook.add(tab, text=f'Tab {i+1}')
        crearTabContenido(tab, num=i + 1)  # Pass tab number



    root.mainloop()

if __name__ == '__main__':
    main()

In [4]:
import tkinter as tk
from tkinter import ttk
from tkinter.filedialog import askopenfilename
from tkinter.filedialog import asksaveasfilename
import soundfile as sf
import sounddevice as sd
import numpy as np

def write_encoded_wav(filename, data, samplerate):
    # data: numpy int32 array
    byte_data = struct.pack('<' + 'i' * len(data), *data)
    with open(filename, 'wb') as out:
        out.write(struct.pack('<4sI4s', b'RIFF', 36 + len(byte_data), b'WAVE'))
        # Format chunk: PCM(1), mono(1), 32 bits/sample
        out.write(struct.pack('<4sIHHIIHH', b'fmt ', 16, 1, 1, samplerate,
                              samplerate * 4, 4, 32))
        out.write(struct.pack('<4sI', b'data', len(byte_data)))
        out.write(byte_data)

def read_encoded_wav(filename):
    with open(filename, 'rb') as f:
        # Read header chunks (simplified)
        riff = f.read(12)
        fmt = f.read(24)
        data_header = f.read(8)
        # Extract sample rate and data size
        samplerate = struct.unpack('<I', fmt[12:16])[0]
        datasize = struct.unpack('<I', data_header[4:8])[0]
        data = f.read(datasize)
        samples = struct.unpack('<' + 'i' * (datasize // 4), data)
        return np.array(samples, dtype=np.int32), samplerate
    
class Audio(tk.Variable):
    def __init__(self):
        self.audio = None
        self.fm = -1

    def get(self):
        return self.audio, self.fm

    def set(self, audio, fm):
        self.audio = audio
        self.fm = fm

def selecAudio(raiz, varAudio):
    varMensaje = tk.StringVar(raiz, 'Selecciona una señal de audio')

    def abrirArchivo():
        fichero = askopenfilename(
            title='Selecciona un fichero de audio',
            filetypes=(('Fichero de audio', '.mp3 .wav'), ('Todos los archivos', '*.*'))
        )
        if fichero:
            varMensaje.set(fichero)
            senyal, fm = sf.read(fichero)
            varAudio.set(senyal, fm)

    boton = ttk.Button(raiz, textvariable=varMensaje, command=abrirArchivo)
    boton.pack()

def reproduce(varAudio, varVolumen):
    audio, fm = varAudio.get()
    if audio is not None:
        sd.play(audio * varVolumen.get(), fm, blocking=False)

def reproduceParaSal(raiz, varAudio, varVolumen):
    marco = ttk.Frame(raiz)
    marco.pack(side=tk.BOTTOM)

    ttk.Button(marco, text='Play', command=lambda: reproduce(varAudio, varVolumen)).pack(side=tk.LEFT)
    ttk.Button(marco, text='Stop', command=sd.stop).pack(side=tk.LEFT)
    ttk.Button(marco, text='Quit', command=raiz.quit).pack(side=tk.LEFT)

def volumen(raiz, varVolumen):
    marco = ttk.LabelFrame(raiz, text='Volumen')
    marco.pack()
    ttk.Scale(marco, from_=0, to=1, length=200, variable=varVolumen).pack()

def EstereoMono(varAudio, varOutput, canal=2):
    import numpy as np

    audio, fm = varAudio.get()
    if audio is None:
        return  # No input audio

    # Confirm it's stereo
    if audio.ndim != 2 or audio.shape[1] != 2:
        raise ValueError("La señal de entrada no es estéreo.")

    L = audio[:, 0]
    R = audio[:, 1]

    if canal == 0:
        mono = L
    elif canal == 1:
        mono = R
    elif canal == 2:
        mono = (L + R) / 2
    elif canal == 3:
        mono = (L - R) / 2
    else:
        raise ValueError("Canal debe ser 0, 1, 2 o 3.")

    # Save mono as 2D array for sounddevice compatibility
    mono = mono.reshape(-1, 1)
    varOutput.set(mono, fm)

def MonoEstereo(varAudio, varOutput):
    import numpy as np

    audio, fm = varAudio.get()
    if audio is None:
        return  # No input audio

    # If already stereo, just pass through or raise error
    if audio.ndim == 2 and audio.shape[1] == 2:
        raise ValueError("La señal ya es estéreo.")

    # Make sure audio is 1D for mono
    if audio.ndim == 2 and audio.shape[1] == 1:
        audio = audio[:, 0]

    # Duplicate mono to stereo by stacking channels
    stereo = np.column_stack((audio, audio))

    varOutput.set(stereo, fm)

def CodEstereo(varAudio, varOutput):
    # Encode stereo audio into int32 coded mono
    audio, fm = varAudio.get()
    if audio is None:
        return
    if audio.ndim != 2 or audio.shape[1] != 2:
        print("Error: input must be stereo audio.")
        return

    int_audio = (audio * 32767).astype(np.int16)
    L = int_audio[:, 0].astype(np.int32)
    R = int_audio[:, 1].astype(np.int32)
    codificats = ((L + R) << 16) | ((L - R) & 0xFFFF)
    varOutput.set(codificats, fm)

def DescoEstereo(varAudio, varOutput):
    # Decode int32 coded mono into stereo audio
    encoded, fm = varAudio.get()
    if encoded is None:
        return
    if not (encoded.dtype == np.int32 and encoded.ndim == 1):
        print("Error: input must be encoded int32 array.")
        return

    def signed_16bit(val):
        return val if val < 0x8000 else val - 0x10000

    vec_signed_16bit = np.vectorize(signed_16bit)
    lower_16 = encoded & 0xFFFF
    lower_16_signed = vec_signed_16bit(lower_16)
    upper_16 = encoded >> 16

    L = ((upper_16 + lower_16_signed) // 2).astype(np.int16)
    R = ((upper_16 - lower_16_signed) // 2).astype(np.int16)
    stereo = np.column_stack((L, R))
    stereo_float = stereo.astype(np.float32) / 32767.0
    varOutput.set(stereo_float, fm)

def botonExtra1(raiz, varAudio, varOutput, num):
    def repartir(varAudio, varOutput, num):
        if num == 1:
            EstereoMono(varAudio, varOutput)
        elif num == 2:
            MonoEstereo(varAudio, varOutput)
        elif num == 3:
            CodEstereo(varAudio, varOutput)
        else:
            DescoEstereo(varAudio, varOutput)

    titulos = {
        1: 'Estéreo a Mono',
        2: 'Mono a Estéreo',
        3: 'Codifica Estéreo',
        4: 'Descodifica Estéreo'
    }
    texto_boton = titulos[num]

    ttk.Button(
        raiz,
        text=texto_boton,
        command=lambda: repartir(varAudio, varOutput, num)
    ).pack(side=tk.LEFT, padx=10)



def botonExtra2(raiz, varOutput):
    def guardarAudio():
        audio, fm = varOutput.get()
        if audio is not None:
            archivo = asksaveasfilename(
                title='Guardar archivo de audio',
                defaultextension='.wav',
                filetypes=(('Archivo WAV', '*.wav'), ('Todos los archivos', '*.*'))
            )
            if archivo:
                sf.write(archivo, audio, fm)
                print(f'Audio guardado en: {archivo}')
        else:
            print("No hay audio para guardar.")

    ttk.Button(raiz, text='Guardar Audio', command=guardarAudio).pack(side=tk.LEFT)

# === Audio player layout per tab ===
def crearTabContenido(tab, num):
    varAudio = Audio()
    varOutput = Audio()
    varVolumen = tk.DoubleVar(tab, 0.6)

    if num in [3, 4]:  # Only for CodEstereo & DecEstereo
        varCodificado = tk.StringVar()
        varDecodificado = tk.StringVar()

    selecAudio(tab, varAudio)

    # Botones extra
    marcoBotonesExtra = ttk.Frame(tab)
    marcoBotonesExtra.pack(pady=10)
    botonExtra1(marcoBotonesExtra, varAudio, varOutput, num)  # Updated
    botonExtra2(marcoBotonesExtra, varOutput)

    volumen(tab, varVolumen)
    reproduceParaSal(tab, varAudio, varVolumen)

# === Main app ===
def main():
    root = tk.Tk()
    root.title("Notebook con Reproductores de Audio")
    root.geometry("600x300")
    root.configure(background='Lavender Blush')
    ttk.Style().configure('.', font=("Arial", 14), background='Lavender Blush', padding=8)

    notebook = ttk.Notebook(root)
    notebook.pack(expand=True, fill='both')

    for i in range(4):
        tab = ttk.Frame(notebook)
        notebook.add(tab, text=f'Tab {i+1}')
        crearTabContenido(tab, num=i + 1)  # Pass tab number

    root.mainloop()

if __name__ == '__main__':
    main()

Error: input must be stereo audio.
No hay audio para guardar.


In [9]:
import tkinter as tk
from tkinter import ttk
from tkinter.filedialog import askopenfilename
from tkinter.filedialog import asksaveasfilename
from tkinter import messagebox
import soundfile as sf
import sounddevice as sd
import numpy as np

def write_encoded_wav(filename, data, samplerate):
    # data: numpy int32 array
    byte_data = struct.pack('<' + 'i' * len(data), *data)
    with open(filename, 'wb') as out:
        out.write(struct.pack('<4sI4s', b'RIFF', 36 + len(byte_data), b'WAVE'))
        # Format chunk: PCM(1), mono(1), 32 bits/sample
        out.write(struct.pack('<4sIHHIIHH', b'fmt ', 16, 1, 1, samplerate,
                              samplerate * 4, 4, 32))
        out.write(struct.pack('<4sI', b'data', len(byte_data)))
        out.write(byte_data)

def read_encoded_wav(filename):
    with open(filename, 'rb') as f:
        # Read header chunks (simplified)
        riff = f.read(12)
        fmt = f.read(24)
        data_header = f.read(8)
        # Extract sample rate and data size
        samplerate = struct.unpack('<I', fmt[12:16])[0]
        datasize = struct.unpack('<I', data_header[4:8])[0]
        data = f.read(datasize)
        samples = struct.unpack('<' + 'i' * (datasize // 4), data)
        return np.array(samples, dtype=np.int32), samplerate
    
class Audio(tk.Variable):
    def __init__(self):
        self.audio = None
        self.params = None
        self.nchannels = None  # NEW

    def cargar(self, ruta):
        with wave.open(ruta, 'rb') as wf:
            self.params = wf.getparams()
            self.nchannels = self.params.nchannels  # store it
            frames = wf.readframes(wf.getnframes())
            self.audio = np.frombuffer(frames, dtype=np.int16)

    def get(self):
        return self.audio, self.fm

    def set(self, audio, fm):
        self.audio = audio
        self.fm = fm

def botonSeleccionarArchivoCodificado(parent, varCodificado):
    def seleccionarArchivoCodificado():
        ruta = askopenfilename(filetypes=[("Archivo codificado", "*.bin")])
        if ruta:
            with open(ruta, 'rb') as f:
                contenido = f.read()
                varCodificado.set(contenido.hex())

    ttk.Button(
        parent,
        text="Seleccionar Archivo Codificado",
        command=seleccionarArchivoCodificado
    ).pack()


def selecAudio(raiz, varAudio):
    varMensaje = tk.StringVar(raiz, 'Selecciona una señal de audio')

    def abrirArchivo():
        fichero = askopenfilename(
            title='Selecciona un fichero de audio',
            filetypes=(('Fichero de audio', '.mp3 .wav'), ('Todos los archivos', '*.*'))
        )
        if fichero:
            varMensaje.set(fichero)
            senyal, fm = sf.read(fichero)
            varAudio.set(senyal, fm)

    boton = ttk.Button(raiz, textvariable=varMensaje, command=abrirArchivo)
    boton.pack()

def reproduce(varAudio, varVolumen):
    audio, fm = varAudio.get()
    if audio is not None:
        sd.play(audio * varVolumen.get(), fm, blocking=False)

def reproduceParaSal(raiz, varAudio, varVolumen):
    marco = ttk.Frame(raiz)
    marco.pack(side=tk.BOTTOM)

    ttk.Button(marco, text='Play', command=lambda: reproduce(varAudio, varVolumen)).pack(side=tk.LEFT)
    ttk.Button(marco, text='Stop', command=sd.stop).pack(side=tk.LEFT)
    ttk.Button(marco, text='Quit', command=raiz.quit).pack(side=tk.LEFT)

def volumen(raiz, varVolumen):
    marco = ttk.LabelFrame(raiz, text='Volumen')
    marco.pack()
    ttk.Scale(marco, from_=0, to=1, length=200, variable=varVolumen).pack()

def EstereoMono(varAudio, varOutput, canal=2):
    import numpy as np

    audio, fm = varAudio.get()
    if audio is None:
        return  # No input audio

    # Confirm it's stereo
    if audio.ndim != 2 or audio.shape[1] != 2:
        raise ValueError("La señal de entrada no es estéreo.")

    L = audio[:, 0]
    R = audio[:, 1]

    if canal == 0:
        mono = L
    elif canal == 1:
        mono = R
    elif canal == 2:
        mono = (L + R) / 2
    elif canal == 3:
        mono = (L - R) / 2
    else:
        raise ValueError("Canal debe ser 0, 1, 2 o 3.")

    # Save mono as 2D array for sounddevice compatibility
    mono = mono.reshape(-1, 1)
    varOutput.set(mono, fm)

def MonoEstereo(varAudio, varOutput):
    import numpy as np

    audio, fm = varAudio.get()
    if audio is None:
        return  # No input audio

    # If already stereo, just pass through or raise error
    if audio.ndim == 2 and audio.shape[1] == 2:
        raise ValueError("La señal ya es estéreo.")

    # Make sure audio is 1D for mono
    if audio.ndim == 2 and audio.shape[1] == 1:
        audio = audio[:, 0]

    # Duplicate mono to stereo by stacking channels
    stereo = np.column_stack((audio, audio))

    varOutput.set(stereo, fm)

def CodEstereo(varAudio, varCodificado):
    if varAudio is None or varAudio.audio is None or varAudio.params.nchannels != 2:
        messagebox.showerror("Error", "Input must be stereo audio.")
        return

    L = varAudio.audio[::2]
    R = varAudio.audio[1::2]

    codificats = [((int(l) + int(r)) << 16) | ((int(l) - int(r)) & 0xFFFF) for l, r in zip(L, R)]
    byte_data = struct.pack('<' + 'i' * len(codificats), *codificats)
    
    varCodificado.set(byte_data.hex())  # Store as hex string for safety inside StringVar
    messagebox.showinfo("Codificación", "Codificación completada correctamente.")


def DecEstereo(varCodificado, varOutput):
    if not varCodificado.get():
        messagebox.showwarning("Advertencia", "No hay datos codificados cargados.")
        return
    
    byte_data = bytes.fromhex(varCodificado.get())
    codificats = struct.unpack('<' + 'i' * (len(byte_data) // 4), byte_data)

    L = [((x >> 16) + ((x & 0xFFFF) if (x & 0x8000) == 0 else (x & 0xFFFF) - 0x10000)) // 2 for x in codificats]
    R = [((x >> 16) - ((x & 0xFFFF) if (x & 0x8000) == 0 else (x & 0xFFFF) - 0x10000)) // 2 for x in codificats]

    intercalado = np.empty(len(L) + len(R), dtype=np.int16)
    intercalado[::2] = L
    intercalado[1::2] = R

    varOutput.audio = intercalado
    varOutput.sample_rate = 44100  # We might read actual sample_rate if we store header
    varOutput.nchannels = 2
    messagebox.showinfo("Decodificación", "Decodificación completada correctamente.")


def botonExtra1(raiz, varAudio, varOutput, num):
    def repartir(varAudio, varOutput, num):
        if num == 1:
            EstereoMono(varAudio, varOutput)
        elif num == 2:
            MonoEstereo(varAudio, varOutput)
        elif num == 3:
            CodEstereo(varAudio, varOutput)
        else:
            DescoEstereo(varAudio, varOutput)

    titulos = {
        1: 'Modificació',
        2: 'Modificació',
        3: 'Codificar',
        4: 'Descodificar'
    }
    texto_boton = titulos[num]

    ttk.Button(
        raiz,
        text=texto_boton,
        command=lambda: repartir(varAudio, varOutput, num)
    ).pack(side=tk.LEFT, padx=10)

def botonGuardarArchivoCodificado(parent, varCodificado):
    def guardar():
        if not varCodificado.get():
            messagebox.showwarning("Advertencia", "No hay datos codificados para guardar.")
            return
        
        ruta = asksaveasfilename(defaultextension=".bin", filetypes=[("Archivo codificado", "*.bin")])
        if ruta:
            with open(ruta, 'wb') as f:
                f.write(bytes.fromhex(varCodificado.get()))
            messagebox.showinfo("Guardado", "Archivo codificado guardado correctamente.")

    ttk.Button(parent, text="Guardar Archivo Codificado", command=guardar).pack(side=tk.LEFT)



def botonExtra2(raiz, varOutput):
    def guardarAudio():
        audio, fm = varOutput.get()
        if audio is not None:
            archivo = asksaveasfilename(
                title='Guardar archivo de audio',
                defaultextension='.wav',
                filetypes=(('Archivo WAV', '*.wav'), ('Todos los archivos', '*.*'))
            )
            if archivo:
                sf.write(archivo, audio, fm)
                print(f'Audio guardado en: {archivo}')
        else:
            print("No hay audio para guardar.")

    ttk.Button(raiz, text='Guardar Audio', command=guardarAudio).pack(side=tk.LEFT)

# === Audio player layout per tab ===
def crearTabContenido(tab, num):
    if num == 1:
        varAudio = Audio()
        varOutput = Audio()
        varVolumen = tk.DoubleVar(tab, 0.6)

        selecAudio(tab, varAudio)
        marcoBotonesExtra = ttk.Frame(tab)
        marcoBotonesExtra.pack(pady=10)
        botonExtra1(marcoBotonesExtra, varAudio, varOutput, num)
        botonExtra2(marcoBotonesExtra, varOutput)
        volumen(tab, varVolumen)
        reproduceParaSal(tab, varAudio, varVolumen)

    elif num == 2:
        varAudio = Audio()
        varOutput = Audio()
        varVolumen = tk.DoubleVar(tab, 0.6)

        selecAudio(tab, varAudio)
        marcoBotonesExtra = ttk.Frame(tab)
        marcoBotonesExtra.pack(pady=10)
        botonExtra1(marcoBotonesExtra, varAudio, varOutput, num)
        botonExtra2(marcoBotonesExtra, varOutput)
        volumen(tab, varVolumen)
        reproduceParaSal(tab, varAudio, varVolumen)

    elif num == 3:
        varAudio = Audio()
        varCodificado = tk.StringVar()
        varVolumen = tk.DoubleVar(tab, 0.6)

        selecAudio(tab, varAudio)
        marcoBotonesExtra = ttk.Frame(tab)
        marcoBotonesExtra.pack(pady=10)
        botonExtra1(marcoBotonesExtra, varAudio, varCodificado, num)
        botonGuardarArchivoCodificado(marcoBotonesExtra, varCodificado)
        volumen(tab, varVolumen)
        reproduceParaSal(tab, varAudio, varVolumen)

    elif num == 4:
        varCodificado = tk.StringVar()
        varOutput = Audio()

        botonSeleccionarArchivoCodificado(tab, varCodificado)
        marcoBotonesExtra = ttk.Frame(tab)
        marcoBotonesExtra.pack(pady=10)
        botonExtra1(marcoBotonesExtra, varCodificado, varOutput, num)
        botonExtra2(marcoBotonesExtra, varOutput)


# === Main app ===
def main():
    root = tk.Tk()
    root.title("Notebook con Reproductores de Audio")
    root.geometry("600x300")
    root.configure(background='Lavender Blush')
    ttk.Style().configure('.', font=("Arial", 14), background='Lavender Blush', padding=8)

    notebook = ttk.Notebook(root)
    notebook.pack(expand=True, fill='both')

    tab_tittle = {
        1: 'Estéreo a Mono',
        2: 'Mono a Estéreo',
        3: 'Codifica Estéreo',
        4: 'Descodifica Estéreo'
    }

    for i in range(4):
        tab = ttk.Frame(notebook)
        text_tab = tab_tittle[i+1]
        notebook.add(tab, text=text_tab)
        crearTabContenido(tab, num=i + 1)  # Pass tab number

    root.mainloop()

if __name__ == '__main__':
    main()

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Program Files\Python313\Lib\tkinter\__init__.py", line 2068, in __call__
    return self.func(*args)
           ~~~~~~~~~^^^^^^^
  File "C:\Users\98esq\AppData\Local\Temp\ipykernel_14244\1119137310.py", line 210, in <lambda>
    command=lambda: repartir(varAudio, varOutput, num)
                    ~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\98esq\AppData\Local\Temp\ipykernel_14244\1119137310.py", line 195, in repartir
    CodEstereo(varAudio, varOutput)
    ~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\98esq\AppData\Local\Temp\ipykernel_14244\1119137310.py", line 153, in CodEstereo
    if varAudio is None or varAudio.audio is None or varAudio.params.nchannels != 2:
                                                     ^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'nchannels'


In [10]:
import tkinter as tk
from tkinter import ttk
from tkinter.filedialog import askopenfilename
from tkinter.filedialog import asksaveasfilename
from tkinter import messagebox
import soundfile as sf
import sounddevice as sd
import numpy as np
import wave
import struct

def write_encoded_wav(filename, data, samplerate):
    # data: numpy int32 array
    byte_data = struct.pack('<' + 'i' * len(data), *data)
    with open(filename, 'wb') as out:
        out.write(struct.pack('<4sI4s', b'RIFF', 36 + len(byte_data), b'WAVE'))
        # Format chunk: PCM(1), mono(1), 32 bits/sample
        out.write(struct.pack('<4sIHHIIHH', b'fmt ', 16, 1, 1, samplerate,
                              samplerate * 4, 4, 32))
        out.write(struct.pack('<4sI', b'data', len(byte_data)))
        out.write(byte_data)

def read_encoded_wav(filename):
    with open(filename, 'rb') as f:
        # Read header chunks (simplified)
        riff = f.read(12)
        fmt = f.read(24)
        data_header = f.read(8)
        # Extract sample rate and data size
        samplerate = struct.unpack('<I', fmt[12:16])[0]
        datasize = struct.unpack('<I', data_header[4:8])[0]
        data = f.read(datasize)
        samples = struct.unpack('<' + 'i' * (datasize // 4), data)
        return np.array(samples, dtype=np.int32), samplerate
    
class Audio:
    def __init__(self):
        self.audio = None  # numpy array
        self.fm = None     # sample rate
        self.params = None # wave params (if loaded with wave module)
        self.nchannels = None

    def cargar(self, ruta):
        with wave.open(ruta, 'rb') as wf:
            self.params = wf.getparams()
            self.nchannels = self.params.nchannels
            frames = wf.readframes(wf.getnframes())
            audio = np.frombuffer(frames, dtype=np.int16)
            if self.nchannels == 2:
                audio = audio.reshape(-1, 2)  # convert to 2D array: (num_samples, 2)
            self.audio = audio
            self.fm = wf.getframerate()

    def set(self, audio, fm):
        self.audio = audio
        self.fm = fm
        self.params = None
        self.nchannels = None

    def get(self):
        return self.audio, self.fm

def botonSeleccionarArchivoCodificado(parent, varCodificado):
    def seleccionarArchivoCodificado():
        ruta = askopenfilename(filetypes=[("Archivo codificado", "*.bin")])
        if ruta:
            with open(ruta, 'rb') as f:
                contenido = f.read()
                varCodificado.set(contenido.hex())

    ttk.Button(
        parent,
        text="Seleccionar Archivo Codificado",
        command=seleccionarArchivoCodificado
    ).pack()


def selecAudio(raiz, varAudio):
    varMensaje = tk.StringVar(raiz, 'Selecciona una señal de audio')

    def abrirArchivo():
        fichero = askopenfilename(
            title='Selecciona un fichero de audio',
            filetypes=(('Fichero de audio', '.mp3 .wav'), ('Todos los archivos', '*.*'))
        )
        if fichero:
            varMensaje.set(fichero)
            senyal, fm = sf.read(fichero)
            varAudio.set(senyal, fm)
            varAudio.cargar(fichero)

    boton = ttk.Button(raiz, textvariable=varMensaje, command=abrirArchivo)
    boton.pack()

def reproduce(varAudio, varVolumen):
    audio, fm = varAudio.get()
    if audio is not None:
        sd.play(audio * varVolumen.get(), fm, blocking=False)

def reproduceParaSal(raiz, varAudio, varVolumen):
    marco = ttk.Frame(raiz)
    marco.pack(side=tk.BOTTOM)

    ttk.Button(marco, text='Play', command=lambda: reproduce(varAudio, varVolumen)).pack(side=tk.LEFT)
    ttk.Button(marco, text='Stop', command=sd.stop).pack(side=tk.LEFT)
    ttk.Button(marco, text='Quit', command=raiz.quit).pack(side=tk.LEFT)

def volumen(raiz, varVolumen):
    marco = ttk.LabelFrame(raiz, text='Volumen')
    marco.pack()
    ttk.Scale(marco, from_=0, to=1, length=200, variable=varVolumen).pack()

def EstereoMono(varAudio, varOutput, canal=2):
    import numpy as np

    audio, fm = varAudio.get()
    if audio is None:
        return  # No input audio

    # Confirm it's stereo
    if audio.ndim != 2 or audio.shape[1] != 2:
        raise ValueError("La señal de entrada no es estéreo.")

    L = audio[:, 0]
    R = audio[:, 1]

    if canal == 0:
        mono = L
    elif canal == 1:
        mono = R
    elif canal == 2:
        mono = (L + R) / 2
    elif canal == 3:
        mono = (L - R) / 2
    else:
        raise ValueError("Canal debe ser 0, 1, 2 o 3.")

    # Save mono as 2D array for sounddevice compatibility
    mono = mono.reshape(-1, 1)
    varOutput.set(mono, fm)
    messagebox.showinfo("Cambio", "Archivo Modificado correctamente.")

def MonoEstereo(varAudio, varOutput):
    import numpy as np

    audio, fm = varAudio.get()
    if audio is None:
        return  # No input audio

    # If already stereo, just pass through or raise error
    if audio.ndim == 2 and audio.shape[1] == 2:
        raise ValueError("La señal ya es estéreo.")

    # Make sure audio is 1D for mono
    if audio.ndim == 2 and audio.shape[1] == 1:
        audio = audio[:, 0]

    # Duplicate mono to stereo by stacking channels
    stereo = np.column_stack((audio, audio))

    varOutput.set(stereo, fm)
    messagebox.showinfo("Cambio", "Archivo Modificado correctamente.")

def CodEstereo(varAudio, varCodificado):
    if varAudio is None or varAudio.audio is None or varAudio.params is None:
        messagebox.showerror("Error", "No audio cargado.")
        return

    if varAudio.params.nchannels != 2:
        messagebox.showerror("Error", "El archivo debe ser estéreo (2 canales).")
        return

    # Separate left and right channels
    L = varAudio.audio[::2]
    R = varAudio.audio[1::2]

    codificats = []
    for l, r in zip(L, R):
        sum_lr = int(l) + int(r)
        diff_lr = int(l) - int(r)

        # Saturate sum to 16-bit signed integer range
        sum_lr = max(min(sum_lr, 32767), -32768)

        # Combine into 32-bit word
        combined = (sum_lr << 16) | (diff_lr & 0xFFFF)
        codificats.append(combined)

    # Pack into bytes
    byte_data = struct.pack('<' + 'i' * len(codificats), *codificats)
    varCodificado.set(byte_data.hex())
    messagebox.showinfo("Codificación", "Codificación completada correctamente.")




def DecEstereo(varCodificado, varOutput):
    if not varCodificado.get():
        messagebox.showwarning("Advertencia", "No hay datos codificados cargados.")
        return
    
    byte_data = bytes.fromhex(varCodificado.get())
    codificats = struct.unpack('<' + 'i' * (len(byte_data) // 4), byte_data)

    L = [((x >> 16) + ((x & 0xFFFF) if (x & 0x8000) == 0 else (x & 0xFFFF) - 0x10000)) // 2 for x in codificats]
    R = [((x >> 16) - ((x & 0xFFFF) if (x & 0x8000) == 0 else (x & 0xFFFF) - 0x10000)) // 2 for x in codificats]

    intercalado = np.empty(len(L) + len(R), dtype=np.int16)
    intercalado[::2] = L
    intercalado[1::2] = R

    varOutput.audio = intercalado
    varOutput.sample_rate = 44100  # We might read actual sample_rate if we store header
    varOutput.nchannels = 2
    messagebox.showinfo("Decodificación", "Decodificación completada correctamente.")


def botonExtra1(raiz, varAudio, varOutput, num):
    def repartir(varAudio, varOutput, num):
        if num == 1:
            if varAudio.audio is None:
                messagebox.showerror("Error", "No hay audio cargado.")
                return
            EstereoMono(varAudio, varOutput)
        elif num == 2:
            if varAudio.audio is None:
                messagebox.showerror("Error", "No hay audio cargado.")
                return
            MonoEstereo(varAudio, varOutput)
        elif num == 3:
            if varAudio.audio is None:
                messagebox.showerror("Error", "No hay audio cargado.")
                return
            CodEstereo(varAudio, varOutput)
        else:
            if varAudio.audio is None:
                messagebox.showerror("Error", "No hay audio cargado.")
                return
            DescoEstereo(varAudio, varOutput)

    titulos = {
        1: 'Modificació',
        2: 'Modificació',
        3: 'Codificar',
        4: 'Descodificar'
    }
    texto_boton = titulos[num]

    ttk.Button(
        raiz,
        text=texto_boton,
        command=lambda: repartir(varAudio, varOutput, num)
    ).pack(side=tk.LEFT, padx=10)

def botonGuardarArchivoCodificado(parent, varCodificado):
    def guardar():
        if not varCodificado.get():
            messagebox.showwarning("Advertencia", "No hay datos codificados para guardar.")
            return
        
        ruta = asksaveasfilename(defaultextension=".bin", filetypes=[("Archivo codificado", "*.bin")])
        if ruta:
            with open(ruta, 'wb') as f:
                f.write(bytes.fromhex(varCodificado.get()))
            messagebox.showinfo("Guardado", "Archivo codificado guardado correctamente.")

    ttk.Button(parent, text="Guardar Archivo Codificado", command=guardar).pack(side=tk.LEFT)



def botonExtra2(raiz, varOutput):
    def guardarAudio():
        audio, fm = varOutput.get()
        if audio is not None:
            archivo = asksaveasfilename(
                title='Guardar archivo de audio',
                defaultextension='.wav',
                filetypes=(('Archivo WAV', '*.wav'), ('Todos los archivos', '*.*'))
            )
            if archivo:
                sf.write(archivo, audio, fm)
                print(f'Audio guardado en: {archivo}')
        else:
            print("No hay audio para guardar.")
            messagebox.showinfo("Guardado", "No hay audio para guardar.")

    ttk.Button(raiz, text='Guardar Audio', command=guardarAudio).pack(side=tk.LEFT)

# === Audio player layout per tab ===
def crearTabContenido(tab, num):
    if num == 1:
        varAudio0 = Audio()
        varOutput0 = Audio()
        varVolumen0 = tk.DoubleVar(tab, 0.6)

        selecAudio(tab, varAudio0)
        marcoBotonesExtra = ttk.Frame(tab)
        marcoBotonesExtra.pack(pady=10)
        botonExtra1(marcoBotonesExtra, varAudio0, varOutput0, num)
        botonExtra2(marcoBotonesExtra, varOutput0)
        volumen(tab, varVolumen0)
        reproduceParaSal(tab, varAudio0, varVolumen0)

    elif num == 2:
        varAudio1 = Audio()
        varOutput1 = Audio()
        varVolumen1 = tk.DoubleVar(tab, 0.6)

        selecAudio(tab, varAudio1)
        marcoBotonesExtra = ttk.Frame(tab)
        marcoBotonesExtra.pack(pady=10)
        botonExtra1(marcoBotonesExtra, varAudio1, varOutput1, num)
        botonExtra2(marcoBotonesExtra, varOutput1)
        volumen(tab, varVolumen1)
        reproduceParaSal(tab, varAudio1, varVolumen1)

    elif num == 3:
        varAudio2 = Audio()
        varCodificado2 = tk.StringVar()
        varVolumen2 = tk.DoubleVar(tab, 0.6)

        selecAudio(tab, varAudio2)
        marcoBotonesExtra = ttk.Frame(tab)
        marcoBotonesExtra.pack(pady=10)
        botonExtra1(marcoBotonesExtra, varAudio2, varCodificado2, num)
        botonGuardarArchivoCodificado(marcoBotonesExtra, varCodificado2)
        volumen(tab, varVolumen2)
        reproduceParaSal(tab, varAudio2, varVolumen2)

    elif num == 4:
        varCodificado3 = tk.StringVar()
        varOutput3 = Audio()
        varVolumen3 = tk.DoubleVar(tab, 0.6)

        botonSeleccionarArchivoCodificado(tab, varCodificado3)
        marcoBotonesExtra = ttk.Frame(tab)
        marcoBotonesExtra.pack(pady=10)
        botonExtra1(marcoBotonesExtra, varCodificado3, varOutput3, num)
        botonExtra2(marcoBotonesExtra, varOutput3)
        volumen(tab, varVolumen3)
        reproduceParaSal(tab, varOutput3, varVolumen3)

# === Main app ===
def main():
    root = tk.Tk()
    root.title("Notebook con Reproductores de Audio")
    root.geometry("600x300")
    root.configure(background='Lavender Blush')
    ttk.Style().configure('.', font=("Arial", 14), background='Lavender Blush', padding=8)

    notebook = ttk.Notebook(root)
    notebook.pack(expand=True, fill='both')

    tab_tittle = {
        1: 'Estéreo a Mono',
        2: 'Mono a Estéreo',
        3: 'Codifica Estéreo',
        4: 'Descodifica Estéreo'
    }

    for i in range(4):
        tab = ttk.Frame(notebook)
        text_tab = tab_tittle[i+1]
        notebook.add(tab, text=text_tab)
        crearTabContenido(tab, num=i + 1)  # Pass tab number

    root.mainloop()

if __name__ == '__main__':
    main()

No hay audio para guardar.
No hay audio para guardar.
No hay audio para guardar.
